# Práce s kamerou Basler v Jupyter Notebooku
<img src='images/basler-logo.jpg' style="width:50%">

Vzhledem k tomu, že software Pylon je nejen vlastní program pro připojení a manipulaci s kamerami, ale zároveň SDK, budeme používat přímo SDK. SDK je psáno v C, C++ nebo C#. Basler k tomu však v roce 2018 vytvořil wrapper do Pythonu s názvem `pypylon`.

My jsme k tomu vytvořili rozhraní s názvem `pypylon_opencv_viewer`. To umožňuje se připojit ke konkrétní kameře, zobrazit si snímky z kamery a dále nastavovat některé parametry kamery.

## Nastavení
Import potřebných knihoven

In [1]:
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import cv2
import yaml
from pypylon import pylon 

from IPython.display import Image, display, clear_output

%matplotlib inline

Úvodni nastavení parametrů kamery. Je potřeba dopsat správné **sériové číslo** vlastní kamery. 

V případě, že se nastavuje cesta k ukládání fotek, je třeba nahradit veškerá `\` za `\\`. Při defaultním nastavení se fotky ukládají do složky s tímto ipynb.

In [2]:
#serial_number = '22724647' ### sériové číslo řádkové kamery Ra
serial_number = '21991061' ### sériové číslo maticové kamery Ac
width = 1280
height = 1080
image_folder = '.' ### defaultní nastavení složky pro ukládání snímků

Připojení kamery pomocí `pypylonu`. Je zapotřebí zkontrolovat sériové číslo kamery.

In [3]:
import time
time.sleep(3)

In [4]:
# Pypylon gets camera by serial number
info = None
for i in pylon.TlFactory.GetInstance().EnumerateDevices():
    if i.GetSerialNumber() == serial_number:
        info = i
        break
else:
    print('Camera with {} serial number not found'.format(serial_number))

# VERY IMPORTANT STEP! To use Basler PyPylon OpenCV viewer you have to call .Open() method on you camera
if info is not None:
    camera = pylon.InstantCamera(pylon.TlFactory.GetInstance().CreateDevice(info)) 
    camera.Open()

Pomocná funkce pro zobrazování obrázků

In [5]:
def show_image(img):
    window_name = 'Camera capture (' + serial_number + ')'
    cv2.namedWindow(window_name, cv2.WINDOW_NORMAL | cv2.WINDOW_GUI_NORMAL)
    cv2.resizeWindow(window_name, width, height)
    cv2.imshow(window_name, img)

Vytvoření GUI widgetů pro manipulaci s parametry kamery. Zásady pro tvorbu JSON notace widgetů viz [link](https://github.com/mbalatsko/pypylon-opencv-viewer#list-of-features).

In [6]:
# List of features to create widgets
features = [
    {
        "name": "GainRaw",
        "type": "int"
    },
    {
        "name": "ExposureTimeRaw",
        "type": "int",
        "max": 35000,
        "min": 35,
        "step": 140,
        "value": 3500
    },
    {
        "name": "Height",
        "type": "int_text",
        "max": 1200,
        "min": 100,
        "step": "5"
    },
    {
        "name": "Width",
        "type": "int_text",
        "max": 1920,
        "min": 100,
        "step": "5"
    }
]

In [7]:
import ipywidgets as widgets

In [8]:
features_test = [
    {
        "name": "Height",
        "type": "int_text",
        "unit": "px",
        #"layout": widgets.Layout(width="50%", height="50px")
    },
    {
        "name": "Width",
        "type": "int_text",
        "value": 846,
        "unit": "px",
        #"layout": widgets.Layout(width="50%", height="50px")
    },
    {
        "name": "CenterX",
        "type": "bool",
        #"layout": widgets.Layout(width="10%", height="50px")
    },
    {
        "name": "CenterY",
        "type": "bool",
        #"layout": widgets.Layout(width="10%", height="50px")
    },
    {
        "name": "OffsetX",
        "type": "int",
        "dependency": {"CenterX": False},
        "unit": "px",
        #"layout": widgets.Layout(width="75%", height="50px")
    },
    {
        "name": "OffsetY",
        "type": "int",
        "dependency": {"CenterY": False},
        "unit": "px",
        #"layout": widgets.Layout(width="75%", height="50px")
    },
    {
        "name": "AcquisitionFrameRateAbs",
        "type": "int",
        "unit": "fps",
        "step": 1,
        "max": 150,
        "min": 0,
    },
    {
        "name": "ExposureAuto",
        "type": "choice_text",
        "options": ["Off", "Once", "Continuous"],
        "style": {"button_width": "100px"}
        #"layout": widgets.Layout(width="49%", height="50px")
    },
    {
        "name": "ExposureTimeAbs",
        "type": "int",
        "dependency": {"ExposureAuto": "Off"},
        "unit": "μs",
        "step": 500,
        "max": 35000,
        "min": 500,
        #"layout": widgets.Layout(width="50%", height="50px")
    },
    {
        "name": "BalanceWhiteAuto",
        "type": "choice_text",
        "options": ["Off", "Once", "Continuous"],
        "style": {"button_width": "100px"}
    },
]

Funkce pro image processing. Zatím pouze zobrazuje obrázky.

In [9]:
def impro(img):
    show_image(img)
    return img

## Main
Výkonný kus kódu využívající vlastní knihovnu pro práci s kamerou a knihovnou opencv. Umožňuje nastavení parametrů pomocí GUI widgetů.
- Pro spuštění okna kamery je zapotřebí kliknout na tlačítko `Run Interact`. 
- Okno kamery se vypne stisknutím tlačítka `q`.
- Obrázek se uloží stisknutím tlačítka `s`.
- Pro projevení změn v nastavení pomocí GUI prvků je nutné ukončit okno kamery a znovu ho spustit.

In [10]:
import sys
sys.path
sys.path.insert(0, '/home/kasape/Documents/pypylon-opencv-viewer/')

In [11]:
from pypylon_opencv_viewer import BaslerOpenCVViewer
viewer = BaslerOpenCVViewer(camera, default_user_set="UserSet3")
viewer.set_features(features_test)
#viewer.set_impro_function(impro)
#viewer.run_interaction_single_shot(window_size=(width, height), image_folder=image_folder)
viewer.run_interaction_continuous_shot(window_size=(width, height), image_folder=image_folder)

Output()

In [ ]:
d = {'a': 5, 'b': 7}

In [ ]:
viewer.show()

In [ ]:
widgets.interactive(lambda w: w, {'manual': True}, w=w, ui=ui)

In [ ]:
ui

In [ ]:
type(w)

In [ ]:
from ipywidgets import interact_manual, interact, interactive
import ipywidgets as widgets

items = [widgets.Label(str(i)) for i in range(4)]
widgets.Box(items)

In [ ]:
camera.UserSetSelector.GetValue()